# Machine Learning in Python - Group Project 1

**Due Friday, March 10th by 16.00 pm.**

*include contributors names here (such as Benjamin Chooyin, Name2, ...)*

## General Setup

In [1]:
# Add any additional libraries or submodules below

# Data libraries
import numpy as np
import pandas as pd

# Plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Plotting defaults
plt.rcParams['figure.figsize'] = (8,5)
plt.rcParams['figure.dpi'] = 80

# sklearn modules that are necessary
import sklearn

In [2]:
# Load data
data = pd.read_csv("the_office.csv")
data = pd.DataFrame(data)
data['director']

0           Ken Kwapis
1           Ken Kwapis
2      Ken Whittingham
3         Bryan Gordon
4         Greg Daniels
            ...       
181          Matt Sohn
182       Jesse Peretz
183      Jeffrey Blitz
184       David Rogers
185         Ken Kwapis
Name: director, Length: 186, dtype: object

After making sure that all the necessary libraries or submodules are uploaded here, please follow the given skeleton to create your project report. 
- Your completed assignment must follow this structure 
- You should not add or remove any of these sections, if you feel it is necessary you may add extra subsections within each (such as *2.1. Encoding*). 

**Do not forget to remove the instructions for each section in the final document.**

## 1. Introduction

*This section should include a brief introduction to the task and the data (assume this is a report you are delivering to a client).* 

- If you use any additional data sources, you should introduce them here and discuss why they were included.

- Briefly outline the approaches being used and the conclusions that you are able to draw.

## 2. Exploratory Data Analysis and Feature Engineering

*Include a detailed discussion of the data with a particular emphasis on the features of the data that are relevant for the subsequent modeling.* 

- Including visualizations of the data is strongly encouraged - all code and plots must also be described in the write up. 
- Think carefully about whether each plot needs to be included in your final draft - your report should include figures but they should be as focused and impactful as possible.

*Additionally, this section should also implement and describe any preprocessing / feature engineering of the data.*

- Specifically, this should be any code that you use to generate new columns in the data frame `d`. All of this processing is explicitly meant to occur before we split the data in to training and testing subsets. 
- Processing that will be performed as part of an sklearn pipeline can be mentioned here but should be implemented in the following section.*

**All code and figures should be accompanied by text that provides an overview / context to what is being done or presented.**

In [ ]:
#Exploratory data analysis to get a sense of the variables
print(data.head())
print(data.info)
print(data.describe())

#The data contains 186 episodes with a mean ammount of votes 2130 per episode and a mean rating of 8.25. 
sns.pairplot(data, size = 50)
'''Many of these pairplots contain little information but there are some interesting trends. Season 8 was unpopular.
The ratings are normally distributed, The number of lines and number of words are strongly correlated so we may not need both 
variables. Total votes and imbd rating look like they could have some qaudratic correlation 
'''

   season  episode   episode_name         director  \
0       1        1          Pilot       Ken Kwapis   
1       1        2  Diversity Day       Ken Kwapis   
2       1        3    Health Care  Ken Whittingham   
3       1        4   The Alliance     Bryan Gordon   
4       1        5     Basketball     Greg Daniels   

                                        writer  imdb_rating  total_votes  \
0  Ricky Gervais;Stephen Merchant;Greg Daniels          7.6         3706   
1                                   B.J. Novak          8.3         3566   
2                             Paul Lieberstein          7.9         2983   
3                                Michael Schur          8.1         2886   
4                                 Greg Daniels          8.4         3179   

     air_date  n_lines  n_directions  n_words  n_speak_char  \
0  2005-03-24      229            27     2757            15   
1  2005-03-29      203            20     2808            12   
2  2005-04-05      244       

/opt/conda/lib/python3.9/site-packages/seaborn/axisgrid.py:2076: UserWarning: The `size` parameter has been renamed to `height`; please update your code.
  warnings.warn(msg, UserWarning)


'Many of these pairplots contain little information but there are some interesting trends. Season 8 was unpopular.\nThe ratings are normally distributed, The number of lines and number of words are strongly correlated so we may not need both \nvariables. Total votes and imbd rating look like they could have some qaudratic correlation \n'

In [207]:
#Encoding some of the variables as numerical values instead of strings


#Onehot Encoding the director column

#!!Run these lines only once to convert strings to a list
#for i in range(0,len(data.director)):
#   data.director[i] = [data.director[i]]
for i in range(0,len(data.season)):
   data.season[i] = str(data.season[i])

data = pd.DataFrame(data)

from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

encoded_directors = pd.DataFrame(mlb.fit_transform(data['director']), columns= mlb.classes_)
pd.set_option('display.max_columns', None)
encoded_directors

encoded_seasons = pd.DataFrame(mlb.fit_transform(data['season']), columns= mlb.classes_)
pd.set_option('display.max_columns', None)
print(encoded_seasons)

data.drop()

'''Attempt to one hot encode main characters
#for i in range(0,len(data.main_chars)):
#   data.main_chars[i] = data.main_chars[i].split(';')
mylist = []
for i in range(0,len(data.main_chars)):
    for j in range(0,len(data.main_chars[i])):
        mylist.append(data.main_chars[i][j])
mylist= [set(mylist)]
print(mylist)



#encoded = pd.Data_Frame(mlb.fit_transform([spaced_data["main_chars"]]), columns= mlb.classes_)
#result = pd.concat([data['main_chars'], encoded], axis=1)
#result

'''

     1  2  3  4  5  6  7  8  9
0    1  0  0  0  0  0  0  0  0
1    1  0  0  0  0  0  0  0  0
2    1  0  0  0  0  0  0  0  0
3    1  0  0  0  0  0  0  0  0
4    1  0  0  0  0  0  0  0  0
..  .. .. .. .. .. .. .. .. ..
181  0  0  0  0  0  0  0  0  1
182  0  0  0  0  0  0  0  0  1
183  0  0  0  0  0  0  0  0  1
184  0  0  0  0  0  0  0  0  1
185  0  0  0  0  0  0  0  0  1

[186 rows x 9 columns]


/tmp/ipykernel_67/2185222898.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.season[i] = str(data.season[i])


'Attempt to one hot encode main characters\n#for i in range(0,len(data.main_chars)):\n#   data.main_chars[i] = data.main_chars[i].split(\';\')\nmylist = []\nfor i in range(0,len(data.main_chars)):\n    for j in range(0,len(data.main_chars[i])):\n        mylist.append(data.main_chars[i][j])\nmylist= [set(mylist)]\nprint(mylist)\n\n\n\n#encoded = pd.Data_Frame(mlb.fit_transform([spaced_data["main_chars"]]), columns= mlb.classes_)\n#result = pd.concat([data[\'main_chars\'], encoded], axis=1)\n#result\n\n'

## 3. Model Fitting and Tuning

*In this section you should detail your choice of model and describe the process used to refine and fit that model.*

- You are strongly encouraged to explore many different modeling methods (e.g. linear regression, regression trees, lasso, etc.) but you should not include a detailed narrative of all of these attempts. 
- At most this section should mention the methods explored and why they were rejected - most of your effort should go into describing the model you are using and your process for tuning and validatin it.

*For example if you considered a linear regression model, a classification tree, and a lasso model and ultimately settled on the linear regression approach then you should mention that other two approaches were tried but do not include any of the code or any in depth discussion of these models beyond why they were rejected. This section should then detail is the development of the linear regression model in terms of features used, interactions considered, and any additional tuning and validation which ultimately led to your final model.* 

**This section should also include the full implementation of your final model, including all necessary validation. As with figures, any included code must also be addressed in the text of the document.**

## 4. Discussion and Conclusions


*In this section you should provide a general overview of **your final model**, its **performance**, and **reliability**.* 

- You should discuss what the implications of your model are in terms of the included features, predictive performance, and anything else you think is relevant.

- This should be written with a target audience of a NBC Universal executive who is with the show and university level mathematics but not necessarily someone who has taken a postgraduate statistical modeling course. 

- Your goal should be to convince this audience that your model is both accurate and useful.

- Finally, you should include concrete recommendations on what NBC Universal should do to make their reunion episode a popular as possible.

**Keep in mind that a negative result, i.e. a model that does not work well predictively, but that is well explained and justified in terms of why it failed will likely receive higher marks than a model with strong predictive performance but with poor or incorrect explanations / justifications.**

## 5. References

*In this section, you should present a list of external sources (except the course materials) that you used during the project, if any*

- Additional data sources can be cited here, in addition to related python documentations, any other webpage sources that you benefited from